# colab版MMD自動トレースv2へようこそ！(実行編)

# 始めに

このツールの稼働状況やメンテナンス情報はTwitter（[@miu200521358](https://twitter.com/miu200521358)）にて行っています。

エラーになる、起動しない、などの場合、まずは現在の配布状況をご確認ください。

リプやDM等でのお問い合わせも受け付けています。

また、v2では会員登録後にダウンロードできる学習データが必要なため、会員登録をお願いしています。

# 目次

このノートブックでは、MMD自動トレースの準備と実行を行います。

ノートブックの使い方にはクセがあるので、「[準備編](https://colab.research.google.com/github/miu200521358/motion_trace_colab/blob/master/AutoTraceIntroduction.ipynb)」で使い方に慣れてください。

画面左上の「＝」（実際は三本線）をクリックして下さい。目次が開きます。（既に開いている場合は次へ進んでください）

![目次](https://drive.google.com/uc?export=view&id=1HGk4sJmcPtMbMwcJOvE3aU1GjvKinwA_)

Googleドライブの直下に`autotrace`フォルダを作成してください。

（準備編で既に作られている場合は再作成不要です）

ノートブックを上から順に確認し、以下手順をひとつずつ実行してください。

実行が必要なセルには番号を振ってあります。①から順番に実行してください。


- **「①　データDL」**
  - GoogleドライブおよびGoogleSDK（ツール）と連携します
  - 連携のやり方は、導入編をご確認ください
  - 人物推定に必要な学習データを事前にダウンロードしておきます。
- **「②　環境設定」**
  - ランタイムがGPUに変更できたことを確認します
    - 変更のやり方は、導入編をご確認ください
  - Tensorflowのバージョンを1.xに変更します
  - 効果音を[効果音ラボ](https://soundeffect-lab.info/)様よりダウンロードします
   - 準備や実際のトレース処理等、長い処理時に鳴らします
   - 不要の場合は、ブラウザの音量をミュートにしてください
- **「③　準備」**
    - MMD自動トレースに必要なコードをcolab上に構築します。
    - 大体15分くらいかかります。
- **「④　データ配置」**
    - MMD自動トレースに必要な学習データ等を配置します。
- **「⑤　動画配置」**
    - MMD自動トレースの対象となる映像データを配置します。
- **「⑥　MMD自動トレース実行」**
    - MMD自動トレースを実行します。

---

※作業中に下記のような警告が出ることがありますが、そのまま進めていただいて問題ありません

　（GPUを使うのは主に人物推定とか姿勢推定などの一部分ですので、その他のセルを作業中に出る可能性があります。）

![警告](https://drive.google.com/uc?export=view&id=1mRW32urnPQ4LS4xrLEoPdp_XCqlq1HUF)

---

※下記のようなエラーが出た場合、環境が既に破棄されています。

目次からセル①に戻って、すべて実行し直してください。

![Googleドライブ連携](https://drive.google.com/uc?export=view&id=1Tsjzs2QAECPMfUTaTh7PYX2cMEypEIF2)

# MMD自動トレース実行

## 事前準備

人物推定に必要なデータを2件、それぞれのサイトに会員登録して、ダウンロードしてください。

1. [ExPoseプロジェクトページ](https://expose.is.tue.mpg.de) にアクセスしてください。

2. 「SignIn」ボタンをクリックしてください。

![SignIn](https://drive.google.com/uc?export=view&id=1__io7EjqFQnhq08Zi_YfxKUeSottZWJl)

3. 「Register」ボタンをクリックしてください。

![SignIn](https://drive.google.com/uc?export=view&id=1B8mJPJuTdHANo0UJ71Gue6CmznSDCdbw)

4. 「メールアドレス」「パスワード」を入力して、スイッチをすべてONにして、「Register」ボタンをクリックしてください。

![SignIn](https://drive.google.com/uc?export=view&id=1R2jCZHKJu71hXLW2KHhSLswDjwJEWEiZ)

5. 届いたメールのリンクをクリックしてください。

6. 登録した「メールアドレス」「パスワード」を入力して、「LogIn」ボタンをクリックしてください。

![SignIn](https://drive.google.com/uc?export=view&id=1loay_bPetEBQfaOQIpS_JCLhZrFHolyF)

7. 「Download」ボタンをクリックしてください。

![SignIn](https://drive.google.com/uc?export=view&id=1GFhczKTl5Mc6E6PSRXT0hznCVfo-FWDr)

8. 画面中程の「ExPose Model」ボタンをクリックしてください。

![SignIn](https://drive.google.com/uc?export=view&id=1FjaHb9yGr52nI27xBLHAQBBz4gOEnR6x)

9. ダウンロードしたzipファイル（expose_data.zip）を、`autotrace` フォルダに置いてください。



1. [SMPL-Xプロジェクトページ](https://smpl-x.is.tue.mpg.de) にアクセスしてください。

2. Exposeと同様に会員登録をしてください。

3. 画面上部の「download SMPL-X v1.1」ボタンをクリックしてください。

![SignIn](https://drive.google.com/uc?export=view&id=1cGOHUdhDtSOfIqIHYUXhQTc5ki5pgl85)

4. ダウンロードしたzipファイル（models_smplx_v1_1.zip）を、`autotrace` フォルダに置いてください。


## ①　データDL

Googleドライブの `autotrace` フォルダと連携します。

下の【①-A】のセルを実行して、連携許可のキーを入力欄に「Ctrl+V」で貼り付けて下さい。

### ①-A　Googleドライブとの連携

In [ ]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル①-A】　Googleドライブとの連携

#@markdown ■■■■■■■■■■■■■■■■■■

exec_dict = {}
exec_dict['1'] = True

from google.colab import drive
import os

# Googleドライブマウント
drive.mount('/gdrive')

# 起点ディレクトリ
base_path = "/gdrive/My Drive/autotrace"

if os.path.exists(base_path):
    ! echo "autotraceフォルダの中身 -----------"
    ! ls -l "$base_path"
    ! echo "--------------------"

    print("■■■■■■■■■■■■■■■")
    print("■　** OK **")
    print("■　autotraceフォルダとの連携が成功しました。")
    print("■■■■■■■■■■■■■■■")

    exec_dict['1-A'] = True
else:
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　autotraceフォルダがGoogleドライブの直下に見つかりませんでした。")
    print("■　準備編から実行し直してください。")
    print("■■■■■■■■■■■■■■■")


**【OK】**

下記のように、「autotrace」フォルダの中身が表示されていたら成功です。

![Googleドライブ連携](https://drive.google.com/uc?export=view&id=19OhLvcyP-CN90KWDmkmBuSRR3BpO5GzV)


### ①-B　GoogleSDKとの連携

miuのGoogleドライブから必要なデータをDLするため、`GoogleSDK` と連携します。

セル①-Aと同じく、URLからアクセス許可を与えてください。

下の【①-B】のセルを実行して、連携許可のキーを入力欄に「Ctrl+V」で貼り付けて下さい。

In [ ]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル①-B】　GoogleSDKとの連携

#@markdown ■■■■■■■■■■■■■■■■■■

# Googleドライブアクセスライブラリ
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

if not exec_dict or '1-A' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①-Aが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

exec_dict['1-B'] = True

print("■■■■■■■■■■■■■■■")
print("■　** OK **")
print("■　Google SDK との連携が成功しました。")
print("■■■■■■■■■■■■■■■")


### ①-C　Expose学習データの配置

In [ ]:
#@markdown 下の【①-C】のセルを実行してください。

#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル①-C】　expose_data の配置

#@markdown ■■■■■■■■■■■■■■■■■■

expose_data_path = f"{base_path}/expose_data.zip"

if not exec_dict or '1-A' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①-Aが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '1-B' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①-Bが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not os.path.exists(expose_data_path):
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　autotrace 直下に expose_data.zip が見つかりませんでした。")
    print("■■■■■■■■■■■■■■■")
else:
    if os.path.exists('./expose_data.zip'):
        os.remove('./expose_data.zip')

    ! cp "$expose_data_path" .

    import hashlib
    filechecksum = 'ac8e8a53922364ed7bd04e8f9722f937b987f146f9b53f6b4317a0e69a0d055b'

    with open('./expose_data.zip','rb') as f:
        checksum = hashlib.sha256(f.read()).hexdigest()
    if checksum == filechecksum:
        print("■■■■■■■■■■■■■■■")
        print("■　** OK **")
        print("■　expose_data の配置が成功しました。")
        print("■■■■■■■■■■■■■■■")

        exec_dict['1-C'] = True        
    else:
        print("■■■■■■■■■■■■■■■")
        print("■　** ERROR **")
        print("■　autotraceフォルダの expose_data のハッシュ値が一致しません。")
        print("■　配置に失敗している可能性があるので、もう一度セルを実行し直してください。")
        print("■　3回実行してもエラーになる場合、アップロードからやり直して下さい。")
        print("■■■■■■■■■■■■■■■")


### ①-D　SMPL-X学習データの配置


In [ ]:
#@markdown 下の【①-D】のセルを実行してください。

#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル①-D】　SMPL-Xデータ の配置

#@markdown ■■■■■■■■■■■■■■■■■■

smplx_data_path = f"{base_path}/models_smplx_v1_1.zip"

if not exec_dict or '1-A' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①-Aが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '1-B' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①-Bが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '1-C' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①-Cが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not os.path.exists(smplx_data_path):
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　autotrace 直下に models_smplx_v1_1.zip が見つかりませんでした。")
    print("■■■■■■■■■■■■■■■")
    
else:
    if os.path.exists('./models_smplx_v1_1.zip'):
        os.remove('./models_smplx_v1_1.zip')

    ! cp "$smplx_data_path" .

    import hashlib
    filechecksum = 'cb593838a5d602395735081c7a8fd1d6b04fba261042b84e24644d875039be61'

    with open('./models_smplx_v1_1.zip','rb') as f:
        checksum = hashlib.sha256(f.read()).hexdigest()
    if checksum == filechecksum:
        print("■■■■■■■■■■■■■■■")
        print("■　** OK **")
        print("■　models_smplx_v1_1 の配置が成功しました。")
        print("■■■■■■■■■■■■■■■")
        exec_dict['1-D'] = True
    else:
        print("■■■■■■■■■■■■■■■")
        print("■　** ERROR **")
        print("■　autotraceフォルダの models_smplx_v1_1 のハッシュ値が一致しません。")
        print("■　配置に失敗している可能性があるので、もう一度セルを実行し直してください。")
        print("■　3回実行してもエラーになる場合、アップロードからやり直して下さい。")
        print("■■■■■■■■■■■■■■■")


## **②　環境設定**

ヘッダの "ランタイム"　＞　"ランタイムのタイプを変更"　＞　"GPU"　を選択して下さい。

変更できたら、下の【②】のセルを実行して下さい。

In [ ]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル②】　
#@markdown - ランタイムがGPUであることの確認
#@markdown - Tensorflowのバージョンを 1.x に変更
#@markdown - 効果音を[効果音ラボ](https://soundeffect-lab.info/)様よりダウンロード

#@markdown ■■■■■■■■■■■■■■■■■■

if not exec_dict or '1-A' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①-Aが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '1-B' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①-Bが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '1-C' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①-Cが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '1-D' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①-Dが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

exec_dict['2'] = True

! echo --------------
! echo 【A】 ランタイムをGPUに変更
! echo --------------

! nvcc --version
! nvidia-smi

! echo --------------
! echo 【B】 Tensorflowのバージョンを 1.x に変更
! echo --------------

%tensorflow_version 1.x

%tensorflow_version

! echo --------------
! echo 【D】 効果音をダウンロード
! echo --------------

! wget --no-check-certificate -c "https://soundeffect-lab.info/sound/anime/mp3/sceneswitch1.mp3"

from IPython.display import Audio, display
def play_sound():
    try:
        display(Audio("sceneswitch1.mp3", autoplay=True))
    except:
        print("■■■■■■■■■■■■■■■")
        print("■　効果音が再生できませんでした")
        print("■■■■■■■■■■■■■■■")

play_sound()


**【OK】**

下記のように表示されて、最後に音が鳴ったら成功です。

![GPU切り替え失敗](https://drive.google.com/uc?export=view&id=1WK0Oo3qQESB8hVlkvyWfsq17LUn3N308)

---

**【NG】**

下記のように表示されていたら、ランタイムの変更に失敗しているので、導入編を再度確認して、ランタイムを変更してください。

![GPU変更成功](https://drive.google.com/uc?export=view&id=1E230eOWMctbD2F7qTvn_fR-3gBRQQ_uQ)


## **③　コード準備**

MMD自動トレースコードをColab上に構築します。

下の【③】のセルを実行してください。たくさんメッセージが出ます。15分くらいかかります。

**途中でコンパイルが走るので、動かなくなったように見える箇所があります。**

**ビックリしたり心配になって、セルを中断しないようお願いします。そのまま待っていれば終わります。**

`ERROR` という文字列が出る場合もありますが、自動トレースを実行するには問題ないエラーの可能性が高いので、一旦無視してそのまま進めて下さい。

（2021/08/15時点ではERRORが1行、WARNINGが2行出ますが、どれもトレースに支障はありません。

　それ以上ERRORが出た場合、インストールに失敗してる可能性が高いので、セル③をやり直してください）

【②】でDLした音が鳴ったら終了です。

In [ ]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル③】　コード準備実行

#@markdown ■■■■■■■■■■■■■■■■■■

if not exec_dict or '1-A' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①-Aが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '1-B' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①-Bが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '1-C' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①-Cが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '2' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル②が実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

else:
    import shutil
    
    if os.path.exists('./expose_mmd'):
        shutil.rmtree('./expose_mmd')

    # MMD自動トレースキットバージョンタグ
    ver_tag = "ver2.05.02.00"

    ! git clone  --depth 1 -b "$ver_tag" "https://github.com/miu200521358/expose_mmd.git"

    ! pip install --upgrade -r "expose_mmd/requirements.txt"

    import os 
    os.environ["CUDA_HOME"]="/usr/local/cuda"
    os.environ["LD_LIBRARY_PATH"]="/usr/local/cuda/lib64"

    ! cd expose_mmd/lighttrack/lib/ && python setup.py install
    ! cd expose_mmd/lighttrack/graph/torchlight && python setup.py install

    exec_dict['3'] = True

    print("■■■■■■■■■■■■■■■")
    print("■　** OK **")
    print("■　コードの構築が完了しました。")
    print("■■■■■■■■■■■■■■■")

    play_sound()

## **④　データ配置**

MMD自動トレースに必要なデータをColab上に配置します。

下の【④】のセルを実行してください。たくさんメッセージが出ます。

【②】でDLした音が鳴ったら終了です。

In [ ]:

#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル④】　データの配置

#@markdown ■■■■■■■■■■■■■■■■■■

if not exec_dict or '1-A' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①-Aが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '1-B' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①-Bが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '1-C' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①-Cが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '2' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル②が実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '3' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル③が実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

else:
    import os
    import shutil
    if os.path.exists('./data'):
        shutil.rmtree('./data')

    if os.path.exists('./models'):
        shutil.rmtree('./models')

    if os.path.exists('./expose_mmd/data/'):
        shutil.rmtree('./expose_mmd/data/')

    if os.path.exists('./expose_mmd/lighttrack/weights/'):
        shutil.rmtree('./expose_mmd/lighttrack/weights/')
    os.makedirs('./expose_mmd/lighttrack/weights/', exist_ok=True)

    # expose関連データ移動
    ! unzip ./expose_data.zip && mv -f data ./expose_mmd/
    ! unzip ./models_smplx_v1_1.zip && mv -f models ./expose_mmd/data/

    # backbones.zip
    downloaded = drive.CreateFile({'id': '17Pq_Yjx2TaZeSVxE1kLkhTs8UlDgAAMd'})
    downloaded.GetContentFile('/content/backbones.zip')
    ! mv -f ./backbones.zip expose_mmd/lighttrack/weights/ && cd expose_mmd/lighttrack/weights && unzip -o ./backbones.zip

    # CPN101.zip
    downloaded = drive.CreateFile({'id': '10pOvWra9GysGJQxEhOj4j5VsOIWRFtlm'})
    downloaded.GetContentFile('/content/CPN101.zip')
    ! mv -f ./CPN101.zip expose_mmd/lighttrack/weights/ && cd expose_mmd/lighttrack/weights && unzip -o ./CPN101.zip

    # GCN.zip
    downloaded = drive.CreateFile({'id': '1KuL2uLfu-GLZLFHq7XKjjJWOjE_18Vzn'})
    downloaded.GetContentFile('/content/GCN.zip')
    ! mv -f ./GCN.zip expose_mmd/lighttrack/weights/ && cd expose_mmd/lighttrack/weights && unzip -o ./GCN.zip

    # mobile-deconv.zip
    downloaded = drive.CreateFile({'id': '11cMlBVuNJon_2XjXhFTXGLZXAO32k7Q3'})
    downloaded.GetContentFile('/content/mobile-deconv.zip')
    ! mv -f ./mobile-deconv.zip expose_mmd/lighttrack/weights/ && cd expose_mmd/lighttrack/weights && unzip -o ./mobile-deconv.zip

    # MSRA152.zip
    downloaded = drive.CreateFile({'id': '14Yz3Z_5avWFxu8MZ7vhWP2M-xPxUEQKh'})
    downloaded.GetContentFile('/content/MSRA152.zip')
    ! mv -f ./MSRA152.zip expose_mmd/lighttrack/weights/ && cd expose_mmd/lighttrack/weights && unzip -o ./MSRA152.zip

    # YOLOv3.zip
    downloaded = drive.CreateFile({'id': '1s9Rs90jKrlCvwM9oWgituKlo-SmPhp4q'})
    downloaded.GetContentFile('/content/YOLOv3.zip')
    ! mv -f ./YOLOv3.zip expose_mmd/lighttrack/weights/ && cd expose_mmd/lighttrack/weights && unzip -o ./YOLOv3.zip

    # # shape_predictor_68_face_landmarks.dat.bz2
    # downloaded = drive.CreateFile({'id': '1Gai5UVpqYJ0aWU9itFPDBu7iy53JqdV6'})
    # downloaded.GetContentFile('/content/shape_predictor_68_face_landmarks.dat.bz2')
    # ! mv -f ./shape_predictor_68_face_landmarks.dat.bz2 expose_mmd/data/ && cd expose_mmd/data && bunzip2 -f ./shape_predictor_68_face_landmarks.dat.bz2

    exec_dict['4'] = True

    print("■■■■■■■■■■■■■■■")
    print("■　** OK **")
    print("■　その他のデータ の配置が成功しました。")
    print("■■■■■■■■■■■■■■■")

    play_sound()

## **⑤　入力映像アップロード**

処理したい映像ファイルを、準備してください。

 - ファイル名は **半角英数字のみ** にしてください。opencvは2バイト文字を読み込めません。
 - あまり長い動画はディスク容量が足りなくなります。6000fを超える場合、分割してください。
 - Googleドライブの **autotrace** フォルダ 直下に置いてください。
 - **マウント後のGooleドライブ上のファイルの上書きや更新は正しく認識されません。** 新しいファイルは新規の名前でアップロードしてから処理して下さい。
 - 入力するのは、ファイル名（拡張子含む）のみです。
 - 下の【⑤】のセルを実行して下さい。

In [ ]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル⑤】　入力映像ファイルアップロード

#@markdown ■■■■■■■■■■■■■■■■■■


#@markdown 解析対象となる映像のファイルの名前を入力して、セルを実行してください。

input_video_name = "input.mp4"  #@param {type: "string"}

if not exec_dict or '1-A' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①-Aが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '1-B' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①-Bが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '1-C' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①-Cが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '2' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル②が実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '3' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル③が実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '4' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル④が実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

exec_dict['5'] = True

input_video_path = f"{base_path}/{input_video_name}"

if not os.path.exists(input_video_path):
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　autotrace 直下に 指定動画 が見つかりませんでした。")
    print("■　ファイルパス: " + input_video_path)
    print("■■■■■■■■■■■■■■■")
else:
    ! cp "$input_video_path" .

    input_video_path = f"/content/{input_video_name}"

    print("■■■■■■■■■■■■■■■")
    print("■　** OK **")
    print("■　入力動画 の設定が完了しました。")
    print("■　ファイルパス: " + input_video_path)
    print("■■■■■■■■■■■■■■■")

play_sound()

## **⑥　MMD自動トレース実行**

MMD自動トレースは人物推定・追跡指定・モーション出力に分けて実行していただきます。

下の【⑥-A】のセルを実行してください。最後の変換処理で時間がかかります。

※下記メッセージが出てきたら、1回目は無視して大丈夫です。

2回目が出てきたら、入力映像をキリのいいところで区切って分けて処理してください。

![容量オーバー](https://drive.google.com/uc?export=view&id=1EKt3nCK6ZYjgkNoflQNzSnW_0s4WnJYO)




### ⑥-A　人物推定

In [ ]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル⑥-A】　人物推定

#@markdown ■■■■■■■■■■■■■■■■■■

import shutil

if not exec_dict or '1-A' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①-Aが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '1-B' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①-Bが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '1-C' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①-Bが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '2' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル②が実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '3' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル③が実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '4' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル④が実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '5' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル⑤が実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not os.path.exists('/content/expose_mmd/data') or not os.path.exists('/content/expose_mmd/data/models'):
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　学習データの配置に失敗している可能性があります。")
    print("■　①-Cまで戻って、実行し直してください。")
    print("■■■■■■■■■■■■■■■")
    
else:
    if os.path.exists('/content/output'):
        shutil.rmtree('/content/output')

    if os.path.exists('./out.webm'):
        os.remove('./out.webm')

    if os.path.exists('./out2.webm'):
        os.remove('./out2.webm')

    import datetime
    process_img_dir_name = "{0}_{1:%Y%m%d_%H%M%S}".format(os.path.basename(input_video_path).replace('.', '_'), datetime.datetime.now())
    process_img_dir_path = os.path.join('/content/output', process_img_dir_name)
    os.makedirs(process_img_dir_path, exist_ok=True)

    ! cd expose_mmd/ && python executor.py --video-file "$input_video_path" --parent-dir "$process_img_dir_path" --process prepare,expose,tracking --verbose 20 --log-mode 0

    # mp4に変換
    bbox_file_path = f"{process_img_dir_path}/bbox.mp4"

    # from base64 import b64encode
    # mp4 = open(bbox_file_path,'rb').read()
    # data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

    print("■■■■■■■■■■■■■■■")
    print("■　** INFO **")
    print("■　セル⑥-Bを実行するための準備を開始します。")
    print("■　しばらくお待ちください。")
    print("■■■■■■■■■■■■■■■")

    # !ffmpeg -i "$bbox_file_path" -vcodec vp9 ./out.webm
    # ! ffmpeg -i "$bbox_file_path" -r 30 -c:v libvpx-vp9 -threads 8 -speed 1 -tile-columns 6 -frame-parallel 1 -auto-alt-ref 1 -c:a libopus -b:a 64k -f webm ./out.webm
    ! ffmpeg -i "$bbox_file_path" -c:v libvpx-vp9 -b:v 1000K -r 30 -threads 8 -speed 1 -tile-columns 6 -frame-parallel 1 -auto-alt-ref 1 -lag-in-frames 25 -c:a libopus -b:a 64k -f webm -y out.webm

    # # webm形式で別途出力
    # import os
    # import glob
    # import cv2
    # from tqdm import tqdm

    # def save_webm(process_img_pathes, out_path):
    #     fourcc = cv2.VideoWriter_fourcc(*"vp80")
    #     out = cv2.VideoWriter(out_path, fourcc, 30.0, (1280, 720))

    #     for iidx, process_img_path in enumerate(tqdm(process_img_pathes)):
    #         out_frame = cv2.imread(process_img_path)
    #         out.write(out_frame)

    #     out.release()
    #     cv2.destroyAllWindows()

    # process_img_pathes = sorted(glob.glob(os.path.join(process_img_dir_path, "frames", "**", "bbox_*.png")))
    # process_bbox_path = "./out.webm"
    # save_webm(process_img_pathes, process_bbox_path)

    exec_dict['6-A'] = True

    print("■■■■■■■■■■■■■■■")
    print("■　** OK **")
    print("■　セル⑥-Bの実行準備が整いました。次に進んでください。")
    print("■■■■■■■■■■■■■■■")

    play_sound()

⑥-Aで準備した結果を、以下の観点で確認します。

 - 人物であろうと認識された箇所に枠が入ります。
 - 一人の人物を追いかける枠の左上にあるINDEXを確認してください。
 - 現在のフレーム番号は左上に表示しています。
 - 綺麗に人物が取れていないところは、枠が取れていても除外した方が綺麗になります。
    - 体幹がすべて取れていて、ひじひざ辺りまで取れている枠を目安にしてください。
    - 同じ人物に二重に枠がついてしまってる場合、より末端まで取れてる方のみ採用してください。
 - 指定方法
   - INDEXを最初から最後まで取り入れる場合：[INDEX番号]
   - INDEXを最初から途中まで取り入れる場合：[INDEX番号]:-[終了フレーム]
   - INDEXを途中から最後まで取り入れる場合：[INDEX番号]:[開始フレーム]-
   - INDEXを途中から途中まで取り入れる場合：[INDEX番号]:[開始フレーム]-[終了フレーム]

---

**INDEX指定例**

 ```
0,44:263-,94,126,145,148,159,161:-3858,182,244,246
1,123,157,163:3572-,170,180,208:4427-,213,239:5580-
2,44:-261,49,61,64,71,95,135,139,146,156,188,197,205,215,247
 ```

**上記INDEXの読み解き方(一行目：1人目)**

1. `INDEX:0`の枠を最初から最後まで
2. `INDEX:44`の枠の263F以降から最後まで
3. `INDEX:94`の枠を最初から最後まで
3. `INDEX:126`の枠を最初から最後まで
3. `INDEX:145`の枠を最初から最後まで
3. `INDEX:148`の枠を最初から最後まで
3. `INDEX:159`の枠を最初から最後まで
3. `INDEX:161`の枠を最初から3858Fまで
3. `INDEX:182`の枠を最初から最後まで
3. `INDEX:244`の枠を最初から最後まで
3. `INDEX:246`の枠を最初から最後まで

---

下の【⑥-B】のセルを実行してください。

### ⑥-B　人物推定結果確認

In [ ]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル⑥-B】　人物推定結果確認

#@markdown ■■■■■■■■■■■■■■■■■■

exec_dict['6-B'] = True

from IPython.display import HTML
import base64
import io

video = io.open('./out.webm', 'r+b').read()
encoded = base64.b64encode(video)
data  = '''
<video id="out" width="1280" height="720" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>
'''.format(encoded.decode('ascii')) + '''
<br>
<button id="prev-10frame">10F戻る</button>
<button id="prev-1frame">1F戻る</button>
<button id="prev-05frame">0.5F戻る</button>
<button id="next-05frame">0.5F進む</button>
<button id="next-1frame">1F進む</button>
<button id="next-10frame">10F進む</button>
（ズレた時は0.5F移動で調整してください。）
<script type="text/javascript">
    var video = document.getElementById('out');
    var prev05Frame = document.getElementById('prev-05frame');
    var next05Frame = document.getElementById('next-05frame');
    var prev1Frame = document.getElementById('prev-1frame');
    var next1Frame = document.getElementById('next-1frame');
    var prev10Frame = document.getElementById('prev-10frame');
    var next10Frame = document.getElementById('next-10frame');
    var frameRate = 1/30;

    video.onloadedmetadata = function(){
        //next
        next05Frame.addEventListener('click',function(){
            video.currentTime = Math.min(video.duration, video.currentTime + frameRate / 2);
        });

        //prev
        prev05Frame.addEventListener('click',function(){
            video.currentTime = Math.max(0, video.currentTime - frameRate / 2);
        });

        //next
        next1Frame.addEventListener('click',function(){
            video.currentTime = Math.min(video.duration, video.currentTime + frameRate);
        });

        //prev
        prev1Frame.addEventListener('click',function(){
            video.currentTime = Math.max(0, video.currentTime - frameRate);
        });

        //next
        next10Frame.addEventListener('click',function(){
            video.currentTime = Math.min(video.duration, video.currentTime + frameRate * 10);
        });

        //prev
        prev10Frame.addEventListener('click',function(){
            video.currentTime = Math.max(0, video.currentTime - frameRate * 10);
        });
    };
</script>

'''

HTML(data)


In [ ]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown ※推定結果をダウンロードして確認したい場合、このセルを実行してください。

#@markdown Googleドライブに「bbox.mp4」のファイル名でコピー配置します。

#@markdown ■■■■■■■■■■■■■■■■■■

drive_dir_path = os.path.join(base_path, process_img_dir_name)

import os
os.makedirs(drive_dir_path, exist_ok=True)

! cp -f  "$bbox_file_path" "$drive_dir_path"

print("■■■■■■■■■■■■■■■")
print("■　** OK **")
print("■　「bbox.mp4」のGoogleドライブへのコピーが完了しました。")
print("■　ファイルパス: " + drive_dir_path + "/bbox.mp4")
print("■■■■■■■■■■■■■■■")

play_sound()



In [ ]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 追跡指定タイマー

#@markdown ■■■■■■■■■■■■■■■■■■


#@markdown 人物追跡指定を作成する際に、一定時間Colabを触らない時間が出来るかと思います。

#@markdown 20分ほど触ってないと警告が出てしまいます。

#@markdown こちらのセルを実行すると、任意分数のタイマーとなりますので、警告が出なくなります。

timer_minutes = 10  #@param {type: "number"}

#@markdown 「Ctrl+M+I」のキーボードショートカットで、タイマーを途中でストップさせる事が出来ます。

from time import sleep
from tqdm import tqdm

try:
    for _ in tqdm(range(timer_minutes * 60)):
        sleep(1)
except:
    print("")
    print("■■■■■■■■■■■■■■■")
    print("■　** INFO **")
    print("■　タイマーを中断しました。")
    print("■　次のセルを実行してください。")
    print("■■■■■■■■■■■■■■■")

play_sound()

### ⑥-C　人物追跡指定


In [ ]:
#@markdown 人物がひとりずつ動画の最後まで繋がるように、INDEXを指定して下さい。

#@markdown IDが指定されている分だけ出力します。

#@markdown 上から順番に1人目、2人目…となります。（最大10人まで想定）

#@markdown 指定が終わったら、この【⑥-C】のセルを実行してください。このセルは繰り返し実行できます。

#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル⑥-C】　人物追跡指定

#@markdown ■■■■■■■■■■■■■■■■■■

order_1 = "0"  #@param {type: "string"}

order_2 = ""  #@param {type: "string"}

order_3 = ""  #@param {type: "string"}

order_4 = ""  #@param {type: "string"}

order_5 = ""  #@param {type: "string"}

order_6 = ""  #@param {type: "string"}

order_7 = ""  #@param {type: "string"}

order_8 = ""  #@param {type: "string"}

order_9 = ""  #@param {type: "string"}

order_data = [order_1, order_2, order_3, order_4, order_5, order_6, order_7, order_8, order_9]
order_file_path = f'{process_img_dir_path}/order.csv'

with open(order_file_path, 'w') as f:
    for order in order_data:
        f.write(order)
        f.write("\n")

print("■■■■■■■■■■■■■■■")
print("■　** OK **")
print("■　人物INDEXの指定が成功しました。")
print("■　ファイルパス: " + order_file_path)
print("■■■■■■■■■■■■■■■")

exec_dict['6-C'] = True

play_sound()

### ⑥-D　追跡指定実行

In [ ]:
#@markdown 先ほど指定したINDEXで人物追跡を実行します。

#@markdown この【⑥-D】のセルを実行してください。最後の変換処理で時間がかかります。

#@markdown このセルは繰り返し実行できます。

#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル⑥-D】　追跡指定実行

#@markdown ■■■■■■■■■■■■■■■■■■

if not exec_dict or '1-A' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①-Aが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '1-B' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①-Bが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '1-C' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①-Bが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '2' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル②が実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '3' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル③が実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '4' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル④が実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '5' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル⑤が実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '6-A' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル⑥-Aが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '6-B' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル⑥-Bが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '6-C' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル⑥-Cが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not os.path.exists('/content/expose_mmd/data') or not os.path.exists('/content/expose_mmd/data/models'):
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　学習データの配置に失敗している可能性があります。")
    print("■　①-Cまで戻って、実行し直してください。")
    print("■■■■■■■■■■■■■■■")
    
else:
    ! cd expose_mmd/ && python executor.py --img-dir "$process_img_dir_path" --order-file "$order_file_path" --process order --verbose 20 --log-mode 0

    # mp4に変換
    ordered_file_path = f"{process_img_dir_path}/ordered_bbox.mp4"

    # from base64 import b64encode
    # mp4 = open(ordered_file_path,'rb').read()
    # data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

    # ! rm './out2.webm'

    print("■■■■■■■■■■■■■■■")
    print("■　** INFO **")
    print("■　セル⑥-Eを実行するための準備を開始します。")
    print("■　しばらくお待ちください。")
    print("■■■■■■■■■■■■■■■")

    # !ffmpeg -i "$ordered_file_path" -vcodec vp9 ./out2.webm
    # ! ffmpeg -i "$ordered_file_path" -c:v libvpx-vp9 -b:v 1000K -threads 8 -speed 1 -tile-columns 6 -frame-parallel 1 -auto-alt-ref 1 -lag-in-frames 25 -c:a libopus -b:a 64k -f webm ./out2.webm
    ! ffmpeg -i "$ordered_file_path" -c:v libvpx-vp9 -b:v 1000K -r 30 -threads 8 -speed 1 -tile-columns 6 -frame-parallel 1 -auto-alt-ref 1 -lag-in-frames 25 -c:a libopus -b:a 64k -f webm -y out2.webm

    # process_img_pathes = sorted(glob.glob(os.path.join(process_img_dir_path, "frames", "**", "order_*.png")))
    # process_bbox_path = "./out2.webm"
    # save_webm(process_img_pathes, process_bbox_path)

    exec_dict['6-D'] = True

    print("■■■■■■■■■■■■■■■")
    print("■　** OK **")
    print("■　セル⑥-Eの実行準備が整いました。次に進んでください。")
    print("■■■■■■■■■■■■■■■")

    play_sound()

### ⑥-E　追跡指定確認

In [ ]:
#@markdown 意図した通りに人物を追跡出来ているか確認します。

#@markdown この【⑥-E】のセルを実行してください。

#@markdown 想定通りではなかった場合、⑥-Cに戻ってINDEXを指定し直してください。

#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル⑥-E】　追跡指定確認

#@markdown ■■■■■■■■■■■■■■■■■■

exec_dict['6-E'] = True

from IPython.display import HTML
import base64
import io

video = io.open('./out2.webm', 'r+b').read()
encoded = base64.b64encode(video)
data  = '''
<video id="out" width="1280" height="720" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>
'''.format(encoded.decode('ascii')) + '''
<br>
<button id="prev-10frame">10F戻る</button>
<button id="prev-1frame">1F戻る</button>
<button id="prev-05frame">0.5F戻る</button>
<button id="next-05frame">0.5F進む</button>
<button id="next-1frame">1F進む</button>
<button id="next-10frame">10F進む</button>
（ズレた時は0.5F移動で調整してください。）
<script type="text/javascript">
    var video = document.getElementById('out');
    var prev05Frame = document.getElementById('prev-05frame');
    var next05Frame = document.getElementById('next-05frame');
    var prev1Frame = document.getElementById('prev-1frame');
    var next1Frame = document.getElementById('next-1frame');
    var prev10Frame = document.getElementById('prev-10frame');
    var next10Frame = document.getElementById('next-10frame');
    var frameRate = 1/30;

    video.onloadedmetadata = function(){
        //next
        next05Frame.addEventListener('click',function(){
            video.currentTime = Math.min(video.duration, video.currentTime + frameRate / 2);
        });

        //prev
        prev05Frame.addEventListener('click',function(){
            video.currentTime = Math.max(0, video.currentTime - frameRate / 2);
        });

        //next
        next1Frame.addEventListener('click',function(){
            video.currentTime = Math.min(video.duration, video.currentTime + frameRate);
        });

        //prev
        prev1Frame.addEventListener('click',function(){
            video.currentTime = Math.max(0, video.currentTime - frameRate);
        });

        //next
        next10Frame.addEventListener('click',function(){
            video.currentTime = Math.min(video.duration, video.currentTime + frameRate * 10);
        });

        //prev
        prev10Frame.addEventListener('click',function(){
            video.currentTime = Math.max(0, video.currentTime - frameRate * 10);
        });
    };
</script>

'''

HTML(data)


In [ ]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown ※追跡指定結果をダウンロードして確認したい場合、このセルを実行してください。

#@markdown Googleドライブに「ordered_bbox.mp4」のファイル名でコピー配置します。

#@markdown ■■■■■■■■■■■■■■■■■■

drive_dir_path = os.path.join(base_path, process_img_dir_name)

import os
os.makedirs(drive_dir_path, exist_ok=True)

! cp -f  "$ordered_file_path" "$drive_dir_path"

print("■■■■■■■■■■■■■■■")
print("■　** OK **")
print("■　「ordered_bbox.mp4」のGoogleドライブへのコピーが完了しました。")
print("■　ファイルパス: " + drive_dir_path + "/ordered_bbox.mp4")
print("■■■■■■■■■■■■■■■")

play_sound()



### ⑥-F　モーション出力

In [ ]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル⑥-F】　モーション出力

#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 実行したい任意機能にチェックを入れてください。

#@markdown このセルは繰り返し実行できます。

# #@markdown **身体モーション出力**　（表情のみを出力したい場合などはOFFにしてください）

# process_motion_body = True #@param {type:"boolean"}

# #@markdown **上半身のみのモーション出力**　（グルーブを出力しません）

# process_motion_upper = False #@param {type:"boolean"}

#@markdown **手モーション出力**　（やや時間がかかります）

process_motion_hand = False #@param {type:"boolean"}

#@markdown **表情出力**　（そんなに時間はかかりません）

process_motion_face = False #@param {type:"boolean"}

#@markdown チェックできたら、この【⑥-F】のセルを実行してください。

#@markdown Googleドライブの `autotrace`フォルダの下に「`<動画ファイル名>_<実行年月日時分秒（米国時間）>`」のフォルダの下に下記ファイルを出力しているので、それをダウンロードしてください。

#@markdown     - 棒人間モデル（`trace_<実行年月日時分秒（米国時間）>_rot_model_no<追跡人物No>.pmx`）

#@markdown     - 棒人間用モーション（`trace_<実行年月日時分秒（米国時間）>_rot_no<追跡人物No>.vmd`）

#@markdown     - あにまさ式ミク用モーション（`trace_<実行年月日時分秒（米国時間）>_miku_smooth_no<追跡人物No>.vmd`）

#@markdown モーションは、MMDに同梱されている「あにまさ式ミク（初音ミクVer2.pmd）」に準標準プラグインでグルーブ・上半身2ボーンを追加したモデルに準拠しています。

if not exec_dict or '1-A' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①-Aが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '1-B' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①-Bが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '1-C' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①-Bが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '2' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル②が実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '3' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル③が実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '4' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル④が実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '5' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル⑤が実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '6-A' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル⑥-Aが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '6-B' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル⑥-Bが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '6-C' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル⑥-Cが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '6-D' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル⑥-Dが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not exec_dict or '6-E' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル⑥-Eが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

elif not os.path.exists('/content/expose_mmd/data') or not os.path.exists('/content/expose_mmd/data/models'):
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　学習データの配置に失敗している可能性があります。")
    print("■　①-Cまで戻って、実行し直してください。")
    print("■■■■■■■■■■■■■■■")
    
else:
    process_txt = "mediapipe,smooth,motion"

    motion_hand_flg = "1" if process_motion_hand else "0"
    motion_face_flg = "1" if process_motion_face else "0"

    ! cd expose_mmd/ && python executor.py --img-dir "$process_img_dir_path" --order-file "$order_file_path" --process $process_txt --hand-motion $motion_hand_flg --face-motion $motion_face_flg --verbose 20 --log-mode 0

    import shutil

    # 日本語対策でpythonコピー
    output_name = os.path.basename(input_video_name).replace('.', '_')
    output_path = os.path.join(base_path, process_img_dir_name)

    os.makedirs(output_path, exist_ok=True)

    import glob
    for target_file_name in ["trace_*_rot_model_no*.pmx", "trace_*_rot_no*.vmd", "trace_*_miku_smooth_no*.vmd"]:
        for vmd_path in glob.glob(os.path.join(process_img_dir_path, "motion", target_file_name)):
            out_vmd_path = os.path.join(output_path, os.path.basename(vmd_path))
            shutil.copy(vmd_path, out_vmd_path)

    shutil.copy(f'{process_img_dir_path}/ordered_bbox.mp4', output_path)

    exec_dict['6-F'] = True

    print("■■■■■■■■■■■■■■■")
    print("■　** OK **")
    print("■　モーションの出力が成功しました。")
    print("■　ファイルパス: " + output_path)
    print("■■■■■■■■■■■■■■■")

    play_sound()


# 自動トレース後の作業

## ①　GoogleドライブにVMDが出力されない場合

 - **ERROR**　**CRITICAL**　というメッセージが出ていないか確認してください。
 - メッセージが出ていない場合、処理出力フォルダにVMDファイルが出力されているか確認してください。
   1. 画面左のフォルダーボタンをクリックしてください。

 ![フォルダーボタン](https://drive.google.com/uc?export=view&id=1AoTQjFHaz2uY4jGJttmVRkWk2nct2kb3)

   2. `output/<動画ファイル名>_<実行年月日>_<実行時分秒>/motion` フォルダの下にファイルが出来ているか確認してください。

 ![motionフォルダ](https://drive.google.com/uc?export=view&id=1wJsomaHWFwSyt6lmt7Zuli8AJ-cxi3uX)

     - PMXファイルやVMDファイルがある場合、それをダウンロードしてください。
        - 必要なファイルは【⑥-F】に記載してます。
        - ファイルを選択して右クリックすることで、メニューが表示されます。
     - PMXファイルやVMDファイルがない場合、どこかでエラーが発生しています。
     - エラーの原因が分からない場合は、ノートブックを共有してください。
       - 共有方法は、「[準備編](https://colab.research.google.com/github/miu200521358/motion_trace_colab/blob/master/AutoTraceIntroduction.ipynb)」を確認してください。


## ② 　関節位置データが欲しい場合

関節位置データ（姿勢推定・人物追跡・表情認識の結果データ）が必要な場合、

下のセルを実行すると、Googleドライブに該当データを固めたzipがコピーされるので、それをダウンロードしてください。

In [ ]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 生データが欲しい場合、このセルを実行してください。

#@markdown コピーが完了してから、Googleドライブ上で見えるようになるまで数分かかります。

#@markdown ■■■■■■■■■■■■■■■■■■

json_org_path = os.path.join(process_img_dir_path, 'ordered')
zip_name = f'{os.path.basename(process_img_dir_path)}_ordered.zip'
zip_path = os.path.join(process_img_dir_path, zip_name)
drive_zip_path = os.path.join(base_path, zip_name)

! zip -r "$drive_zip_path" "$json_org_path"

# ! echo "$drive_zip_path"
# ! echo "$zip_path"

# ! cp "$zip_path" "$drive_zip_path"

# import shutil
# shutil.copyfile(zip_name, )

# import shutil
# shutil.copytree(f'{process_img_dir_path}/ordered', json_path)

print("■■■■■■■■■■■■■■■")
print("■　** OK **")
print("■　生データのGoogleドライブへのコピーが完了しました。")
print("■　ファイルパス: " + drive_zip_path)
print("■■■■■■■■■■■■■■■")

play_sound()


In [ ]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown スムージング後データが欲しい場合、このセルを実行してください。

#@markdown コピーが完了してから、Googleドライブ上で見えるようになるまで数分かかります。

#@markdown ■■■■■■■■■■■■■■■■■■

json_org_path = os.path.join(process_img_dir_path, 'smooth')
zip_name = f'{os.path.basename(process_img_dir_path)}_smooth.zip'
zip_path = os.path.join(process_img_dir_path, zip_name)
drive_zip_path = os.path.join(base_path, zip_name)

! zip -r "$drive_zip_path" "$json_org_path"

# ! echo "$drive_zip_path"
# ! echo "$zip_path"

# ! cp "$zip_path" "$drive_zip_path"

# import shutil
# shutil.copyfile(zip_name, )

# import shutil
# shutil.copytree(f'{process_img_dir_path}/smooth', json_path)

print("■■■■■■■■■■■■■■■")
print("■　** OK **")
print("■　スムージング後データのGoogleドライブへのコピーが完了しました。")
print("■　ファイルパス: " + drive_zip_path)
print("■■■■■■■■■■■■■■■")

play_sound()

## ③　奥行きやジャンプを修正したい場合

全親で大体の奥行き（センターZ）、横移動量（センターX）を入力した後、[モーションサポーター](https://seiga.nicovideo.jp/seiga/im10581987) の「全親移植」を使用してください。

センターYはグルーブに分けてますので、ジャンプの修正は、手動の方が綺麗になると思います。

## ④　足FKデータを足IKデータに変換したい場合

[モーションサポーター](https://seiga.nicovideo.jp/seiga/im10581987) の「足FKtoIK」を使用してください。

**オススメ設定**

 1. 「足首水平化：チェックON」「かかと・つま先Y=0」で「足FKtoIK」実行


## ⑤　全打ちデータから、不要キーを除きたい場合

[モーションサポーター](https://seiga.nicovideo.jp/seiga/im10581987) の「スムージング」を使用してください。

※スムージングの不要キー削除処理を行うとオリジナルの動きから若干ズレます

**オススメ設定**

 1. 「処理対象：全ボーン・モーフ」「処理回数：4」「補間方法：補間曲線に従う」「不要キー削除処理を追加実行する：チェックON」でスムージング


## ⑥　あにまさ式ミクから他のモデルに合わせたモーションデータに変換したい場合

[VMDサイジング](https://seiga.nicovideo.jp/seiga/im9755721) を使用してください。

**元モデルの作成方法**

1. そぼろ様の [準標準プラグイン](http://www.nicovideo.jp/watch/sm14956092) をPmxEditorに導入する
2. PmxEditorを起動する
3. MMDに同梱されている「初音ミクVer2.pmd」を読み込む
4. 準標準プラグインを実行し、「全ての親」「グルーブ」「上半身2」を導入する

**オススメ設定**

1. 「元モデル：あにまさ式ミク準標準」「スタンス追加補正：ON」「捩り分散：ON」でお好きなモデルに合わせてサイジング

**オススメの作業順番**

1. センター位置調整（あにまさ式ミク：手動）
2. 全親移植③（あにまさ式ミク：モーションサポーター）
3. 足FKtoIK④（あにまさ式ミク：モーションサポーター）
4. スムージング⑤（あにまさ式ミク：モーションサポーター）
3. サイジング⑥（あにまさ式ミク　→　任意のモデル：VMDサイジング）
4. モーション修正（任意のモデル：手動）


## ⑥　別の動画をトレースしたい場合

Colab環境は、90分無操作でなければ、12時間の間は環境が保持されます。

④から別の動画をアップロードしてやり直す事ができます。


※下記のようなエラーが出た場合、環境が既に破棄されています。

目次からセル①に戻って、すべて実行し直してください。

![Googleドライブ連携](https://drive.google.com/uc?export=view&id=1Tsjzs2QAECPMfUTaTh7PYX2cMEypEIF2)




## ⑦　最初からやり直したい場合

1. メニューの「ランタイム」＞「ランタイムを出荷状態にリセット」＞「はい」をクリックしてください。

 ![リセット](https://drive.google.com/uc?export=view&id=1Eyi6OLpndFo63tyJsMtukIiL9p4wqBJs)

2. メニュー右上の「再接続」をクリックしてください。

 ![再接続](https://drive.google.com/uc?export=view&id=1h7jF222tU4ZQz_jIMeDKYb9CNIbHTN1k)

3. 緑のチェックマークがついたらリセット完了です。

 ![リセット完了](https://drive.google.com/uc?export=view&id=1FXq971EddklGtHMHpm_vQM0MtRvQrJ-y)


# ライセンス・クレジット

MMD自動トレースの結果を公開・配布する場合は、必ずライセンスのご確認をお願い致します。Unity等他への適用の場合も同様です。

[MMD自動トレースv2 ライセンス・クレジット](https://github.com/miu200521358/expose_mmd/wiki/MMD自動トレースv2-ライセンス・クレジット)